# Dataset import and preprocessing

**Imports**

In [ ]:
import kagglehub, os, shutil, random, torch, cv2
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from PIL import Image 
import matplotlib.pyplot as plt

**Fixed random seed for reproducability**

In [ ]:
# fix result (adapted from Monshi et al 2021)
# https://github.com/MaramMonshi/CovidXrayNet/blob/main/DataAugmentation/1-resize.ipynb 
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    tf.random.set_seed(seed)
    
SEED = 42
seed_everything(SEED)

**Dataset import**

In [ ]:
# Copy to current directory
target_dir = "chest_xray_data"

if not os.path.exists(target_dir):

    # Download the dataset
    path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

    print("Downloaded to cache:", path)

    shutil.copytree(path, target_dir)
    print(f"Dataset copied to: {target_dir}")
    
else:
    print(f"Dataset already exists at: {target_dir}")

print("\nDataset ready at:", os.path.abspath(target_dir))

### Dataset reshuffle and train, test, val split 80/10/10

Copied from: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia/discussion/485689 

In [ ]:
dataset_path = 'chest_xray_data/chest_xray'
new_dataset_path = 'chest_xray_data_split'

if not os.path.exists(new_dataset_path):
    for split in ['train', 'val', 'test']:
        for cls in ['NORMAL', 'PNEUMONIA']:
            os.makedirs(f'{new_dataset_path}/{split}/{cls}', exist_ok=True)

    for cls in ['NORMAL', 'PNEUMONIA']:
        all_files = []
        for split in ['train', 'val', 'test']:
            source_folder = f'{dataset_path}/{split}/{cls}'
            files = os.listdir(source_folder)
            all_files.extend([(file, source_folder) for file in files])

        random.shuffle(all_files)

        train_files = all_files[:int(len(all_files)*0.8)]
        val_files = all_files[int(len(all_files)*0.8):int(len(all_files)*0.9)]
        test_files = all_files[int(len(all_files)*0.9):]

        for file, source_folder in train_files:
            dest = f'{new_dataset_path}/train/{cls}/{file}'
            shutil.copy(f'{source_folder}/{file}', dest)

        for file, source_folder in val_files:
            dest = f'{new_dataset_path}/val/{cls}/{file}'
            shutil.copy(f'{source_folder}/{file}', dest)

        for file, source_folder in test_files:
            dest = f'{new_dataset_path}/test/{cls}/{file}'
            shutil.copy(f'{source_folder}/{file}', dest)  

    print("\nDataset ready at:", os.path.abspath(new_dataset_path))
   
else:
    print(f"Dataset already exists at: {new_dataset_path}")

**Find the smallest and largest image dimensions present in the dataset** \
Used to investigate variation in size

In [ ]:
dataset_path = 'chest_xray_data_split'
sizes = []

for split in ['train', 'val', 'test']:
    for cls in ['NORMAL', 'PNEUMONIA']:
        folder = f'{dataset_path}/{split}/{cls}'
        for filename in os.listdir(folder):
            img = Image.open(os.path.join(folder, filename))
            width, height = img.size
            sizes.append((width * height, width, height, filename))

sizes.sort()  # Sort from smallest to largest

print("Top 5 smallest images:")
for i in range(5):
    pixels, w, h, name = sizes[i]
    print(f"{name}: {w} x {h} ({pixels:,} pixels)")


sizes.sort(reverse=True)  # Sort from largest to smallest

print("\nTop 5 largest images:")
for i in range(5):
    pixels, w, h, name = sizes[i]
    print(f"{name}: {w} x {h} ({pixels:,} pixels)")

**Data preprocessing**

In [ ]:
mean = 0.5
std = 0.5

# Define the transforms to do for training data
train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((480, 480)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Define the transforms to do for validation and testing data
val_test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((480, 480)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Load datasets with progress bar
def load_data(path, train_aug=True):
    transform = train_transform if train_aug else val_test_transform
    dataset = datasets.ImageFolder(path, transform=transform)
    
    images = []
    labels = []
    for i in tqdm(range(len(dataset)), desc=f"Loading {path.split('/')[-1]}"):
        img, label = dataset[i]
        images.append(img.numpy().squeeze())
        labels.append(label)
    
    return np.array(images), np.array(labels)

X_train, y_train = load_data('chest_xray_data_split/train', train_aug=True)
X_val, y_val = load_data('chest_xray_data_split/val', train_aug=False)
X_test, y_test = load_data('chest_xray_data_split/test', train_aug=False)

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

**Visualization of data**

In [ ]:
def denormalize(img):
    return np.clip(img * 0.5 + 0.5, 0, 1)

# Get 4 samples from each class
normal_indices = np.where(y_train == 0)[0][:4]
pneumonia_indices = np.where(y_train == 1)[0][:4]
indices = np.concatenate([normal_indices, pneumonia_indices])

fig, axes = plt.subplots(2, 4, figsize=(12, 6))
for i in range(8):
    row, col = i // 4, i % 4
    idx = indices[i]
    img = denormalize(X_train[idx])
    axes[row, col].imshow(img, cmap='gray')
    axes[row, col].set_title(['NORMAL', 'PNEUMONIA'][y_train[idx]])
    axes[row, col].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
def denormalize(img):
    return np.clip(img * 0.5 + 0.5, 0, 1)

# Get 2 samples from each class
normal_indices = np.where(y_val == 0)[0][:2]
pneumonia_indices = np.where(y_val == 1)[0][:2]
indices = np.concatenate([normal_indices, pneumonia_indices])

fig, axes = plt.subplots(2, 2, figsize=(8, 8))
for i in range(4):
    row, col = i // 2, i % 2
    idx = indices[i]
    img = denormalize(X_val[idx])
    axes[row, col].imshow(img, cmap='gray')
    axes[row, col].set_title(['NORMAL', 'PNEUMONIA'][y_train[idx]])
    axes[row, col].axis('off')
plt.tight_layout()
plt.show()